In [1]:
import db_conn
import pandas as pd
import re
pd.options.display.max_columns = None

/home/grace/.local/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = db_conn.get_connection()

## 1. 추출된 전체 부작용
추출된 전체 부작용 중 식별이 가능한 UNIQUE한 약물만을 추려냈습니다.

In [3]:
def get_total_ads():
    cur = conn.cursor()
    cur.execute("""
    SELECT DISTINCT(TRIM(LOWER(d.tt_ad))) AS t_ad
    FROM
    (SELECT id, tt_ad
    FROM 
        (SELECT * 
        FROM article_table_sentences_m
        WHERE table_id IN 
        (SELECT id FROM article_tables WHERE final_topic IS NOT NULL)) M,
        UNNEST(STRING_TO_ARRAY(t_ad, ' , ')) AS tt_ad
    WHERE (tt_ad='') is false) as d;
    """)
    return cur.fetchall()

def get_ad_dictionary():
    cur = conn.cursor()
    cur.execute("""
    SELECT llt_code, pt_code, TRIM(LOWER(llt_name)) AS t_ad
    FROM meddra_llt_181022 M;
    """)
    return cur.fetchall()

In [4]:
ads = pd.DataFrame(get_total_ads())
# ads = pd.read_csv('./distinct_ads.tsv', sep='\t')

In [5]:
# ads.to_csv('./ads.csv', sep=',')

In [6]:
ads.columns=['name']

In [7]:
ads['refined'] = ''
ads.head()

,name,refined
0,odynophagia,
1,presyncope,
2,myalgia,
3,seizure,
4,aphthous ulcer,


#### 전처리 전 총 부작용의 수는 4900개 입니다

In [8]:
ads.shape

(4900, 2)

### 2. 전처리

#### A. dictionary를 참고하여, 정확히 일치하는 부작용 명은 전처리 없이 포함합니다.

In [9]:
dictionary = pd.DataFrame(get_ad_dictionary())
dictionary.columns = ['llt_code', 'pt_code', 't_ad']
# dictionary = pd.read_csv('./dictionary_ads.tsv', sep='\t')
# dictionary = list(map(lambda x: str(x[0]).lower().replace('"', ''), dictionary))

In [10]:
dictionary.shape

(60939, 3)

In [11]:
dictionary.head()

,llt_code,pt_code,t_ad
0,10000002,10000002,11-beta-hydroxylase deficiency
1,10000007,10000007,17 ketosteroids urine decreased
2,10000010,10000007,17 ketosteroids urine low
3,10000008,10000009,17 ketosteroids urine high
4,10000009,10000009,17 ketosteroids urine increased


### 부작용용어 dictionary와 정확이 매칭하면 추가 전처리 없이 확정

In [12]:
def apply_dictionary():
    for ind, ad in ads.iterrows():
    #     print(ad['name'].strip())
        name = ad['name'].strip()
        exact_mathing = dictionary.loc[dictionary.t_ad==name]

        if len(exact_mathing)==0:
            if name[0] in ['"', '.', ':', '-', '(', ')']:
                name = name[1:]
            if name[len(name)-1] in ['"', '.', ':', '-', '(', ')']:
                name = name[:len(name)-1]
    #         print(name)

            exact_mathing = dictionary.loc[dictionary.t_ad==name]

        if len(exact_mathing)>0:
            ads.loc[ind]['refined'] = name

apply_dictionary()

In [13]:
ads[ads['refined'] != ''].shape

(3993, 2)

#### A. 부작용을 포함하고 있는 구절이 추출 된 경우, 부작용 외적인 단어들을 제거합니다

## 선생님께서 need_to_refined_ads_hyj.xlsx파일을 통해 부작용명이 아니라고 지정해주신 목록을 추가하였습니다.

In [14]:
replace_with_space = ['beverages', '<70%', '%', 'values)', '(', ')', '-1987', '-1993', ':', '±', '↑', 'australia-new',
                     'scale-7', 'c-statistics', '-defined', '-derived', '-like', 'like', 'non-beverage',
                     'scale-0', '-days', 'fluorine-18-labelled', '-specific', 'co-variate', '-guided', '≥2°',
                     '≥1', 'composting', 'daids', 'decentralised', 'dmards', 'forage', 'full-scale', 'glues',
                     'granger', 'greyscale', 'grey-scale', 'harvesting', 'immersed', 'invariably',
                     '-guidance', 'lifelon', 'mwaura', 'naito', 'nesting', 'nomad', 'nonsystemi', '≥60', 'primers',
                     'prohibited', 'sanger', '-1993', '<50', '<02']

replace_ends_with = ['-related']

In [15]:
rep = dict((re.escape(k), ' ') for k in replace_with_space)
pattern = re.compile("|".join(rep.keys()))

for ind, row in ads.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if refined.strip() == '':
        ads.drop(ind, inplace=True)
    elif row['name'].strip() != refined.strip():
        ads.loc[ind]['refined'] = refined
        print(row['name'])
        print(ads.loc[ind]['refined'])
    
    for end in replace_ends_with:
        if row['name'].endswith(end):
            ads.loc[ind]['refined'] = row['name'].replace(end, '')

ads = ads.reset_index(drop=True)

plasmacytoma-like
plasmacytoma 
hypertension:
hypertension 
cancer-specific
cancer 
serum-sickness-like
serum-sickness 
aids-1993
aids 
migraine-days
migraine 
flu-like illness
flu  illness
melanoma-specific
melanoma 
ischemia-guided
ischemia 
flu-like symptoms
flu  symptoms
pressure-guided
pressure 
influenza-like
influenza 
acne-like
acne 
asthma-like
asthma 
headache-days
headache 
infarction-defined
infarction 
influenza-like symptoms
influenza  symptoms
binge drinking≥1
binge drinking 
(copd)
 copd 
angina-guided
angina 
breast cancer-specific
breast cancer 
flu like symptoms
flu   symptoms
myocardial infarction-defined
myocardial infarction 
aids-1987
aids 
(lbbb)
 lbbb 
lifelong
 g
variable:
variable 
stenosis-specific
stenosis 
influenza like illness
influenza   illness
pressure-derived
pressure 
hypertension-specific
hypertension 


#### B. 약-부작용 관계를 나타는 것이 아닌 경우, 지웁니다

In [16]:
non_drug_ad_rel = ['-free', 'free-', ' free ', 'variable', 'wrong drug administered', 'exercise-induced',
                  'underdose', 'switch', 'massage', 'according', '-defining', 'arizona', 'controller', 'athletics',
                  'atopics', 'average', 'bachelor', 'award', '—gasoline', 'cards', 'comer', 'consisting', 'consumer',
                  'courage', 'covaria', 'coverage', 'dimers', 'distinguish', 'eavalues', 'emilia', 'multivaria',
                  'nsaids', 'preexisting', 'reaches', 'recording', 'restaura', 'spain', 'standard', 'tactics',
                  'tattoos', 'tasmania', 'telcagepant', 'testing', 'tragus', 'post-aids', 'scale–parent', 'body-mass-index',
                  'mass-cavity', 'non-melanoma skin', 'single-blind', 'non-comatose', 'non-wheezers', 'high-pitched',
                  'gadofluorine-m', 'anger-control', 'hiv-1-negative', 'pulmonary-systemic', '1apical', 'pre-infarction',
                  'non-trauma', 'stress-mass-heart', 'self-discharge', 'anti-japanese encephalitis', 'non-hypertension',
                   'non-infarct', 'post-stress', 'injury-donor', 'rate–pressure', 'non-neutropenic infections', 'non-hypertrophic',
                   'anti-tachycardia', 'scale-up', '5-fluorouracil', 'isar-smart', '18f-fluoride', 'post-acute myocardial infarction',
                   'hyper-responsiveness', 'pre-aids', 'mid-cavity', 'post-bleeding', 'scale–cognitive', 'double-blind',
                   'high-pressure', 'post-endocarditis', 'peri-procedural complications', 'modulation-blind', 'pre-shock',
                   'pre-angina', 'diphtheria–pertussis–tetanus', 'australia–new', 'q10values', 'pressure-time', 'non-congenital',
                   'anti-hypertensive', 'sepsis-induced', 'snaith-hamilton', 'mass–body', 'pain-to-balloon', 'pain-to-coronary',
                   'non-prostate cancer', 'cold-induced', 'non-melanoma', 'post-hypoxic', 'pre-injury', 'scale-16', 'non-atopic',
                   'non-hypoxic', 'cancer-norfolk', 'langerhans-cell', 'body-mass', 'double-blinding', 'fluorouracil-based',
                   'scale–child', 'fat-mass', 'nifurtimox-eflornithine', 'no adverse events', 'typical'] #'non'

rep = dict((re.escape(k), ' ') for k in non_drug_ad_rel)
pattern = re.compile("|".join(rep.keys()))

In [17]:
for ind, row in ads.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if row['name'].strip() != refined.strip():
        print(row['name'])
        ads.drop(ind, inplace=True)
        print('===============================')

ads = ads.reset_index(drop=True)

infarction-free
coverage
no adverse events
covariable
atypical
single-blind
genotypically
aids-defining
non-traumatic
multivariate-adjusted
pre-infarction
variable;
post-stress
multivariable–adjusted
switching
non-neutropenic infections
scale-up
isar-smart
post-acute myocardial infarction
hyper-responsiveness
preexisting
atopics
average
post-bleeding
scale–cognitive
bivariable
double-blind
peri-procedural complications
testing
non-melanoma skin
d-dimers
non-congenital
sepsis-induced
mass–body
pain-to-balloon
switched
non-melanoma
switch
body-mass
switchmrk
fluorouracil-based
fat-mass
consisting
telcagepant
distinguish
nifurtimox-eflornithine
atypical pneumonia
metastasis-free
burn—gasoline
multivariate
multivariable
pain-progression-free
courage
univariable
standards
thyroxine free decreased
multivariable+
recording
typically
covaria
massage
non-melanomatous
leukaemia-free
switchback
variable
according
seizure-free
dependent-variable
wrong drug administered
nsaids
“typical”
underdosed


### 파일 중에 부작용이 아니라고 표기해주신 것을 추가로 제거

In [18]:
replace_excel = pd.read_excel('./need_to_refined_ads_hyj.xlsx')
replace_excel.columns = ['key', 'replace_with', '-']
replace_excel = replace_excel[['key', 'replace_with']]
replace_excel.drop_duplicates(inplace=True)
replace_excel.head()

,key,replace_with
0.0,anuloaortic ectasia,annuloaortic ectasia
14.0,epispadias-classical,epispadias
24.0,broncho-pneumonia,broncho pneumonia
40.0,cardio-myopathy,cardiomyopathy
50.0,hypertension-stroke,Hypertension + stroke


In [19]:
replace_excel.shape

(1204, 2)

In [20]:
for ind, cura in replace_excel[(replace_excel.replace_with.str.find('부작용 아님')>=0) | (replace_excel.replace_with.str.find('부작용아님')>=0)].iterrows():
#     print(cura['name'])
    need_to_drop = ads[ads.name.str.lower().str.find(cura['key'].lower())>=0]
    for i_n, d_n in need_to_drop.iterrows():
        ads.drop(i_n, inplace=True)

#### 약-부작용 관계를 나타는 것이 아닌 레코드를 제거하면 3994 남습니다

In [21]:
ads.shape

(3994, 2)

### 복잡 부작용, 스펠링 미스 등을 선생님께서 주신 파일로 정제

In [22]:
def replace_with_manual_curation(ads):
    for ind, row in replace_excel[(replace_excel.replace_with.str.find('부작용 아님')<0) & (replace_excel.replace_with.str.find('부작용아님')<0)].iterrows():
        print(row)
        for d_ind, d_row in ads.loc[ads.name.str.strip()==row['key']].iterrows():
            origin_str = d_row['name'].replace(row['key'], row['replace_with'].lower())

            ads.loc[d_ind]['refined'] = origin_str 
    return ads

ads = replace_with_manual_curation(ads)

key              anuloaortic ectasia
replace_with    annuloaortic ectasia
Name: 0.0, dtype: object
key             epispadias-classical
replace_with              epispadias
Name: 14.0, dtype: object
key             broncho-pneumonia
replace_with    broncho pneumonia
Name: 24.0, dtype: object
key             cardio-myopathy
replace_with     cardiomyopathy
Name: 40.0, dtype: object
key               hypertension-stroke
replace_with    Hypertension + stroke
Name: 50.0, dtype: object
key             mgh-adhd
replace_with        ADHD
Name: 64.0, dtype: object
key             heat-shock
replace_with         shock
Name: 77.0, dtype: object
key             neoplasm-associated
replace_with               neoplasm
Name: 86.0, dtype: object
key               neoangiogenesis-inflammation
replace_with    Neoangiogenesis + inflammation
Name: 114.0, dtype: object
key               dementia-motor neuron disease
replace_with    Dementia + motor neuron disease
Name: 132.0, dtype: object
key             v

Name: 2152.0, dtype: object
key             wisckott-aldrich syndrome
replace_with     Wiskott-Aldrich syndrome
Name: 2156.0, dtype: object
key             anxiety-withdrawal
replace_with               anxiety
Name: 2181.0, dtype: object
key             pre-eclampsiadelivery
replace_with            pre-eclampsia
Name: 2204.0, dtype: object
key             ventricular–dilated cardiomyopathy
replace_with                dilated cardiomyopathy
Name: 2225.0, dtype: object
key                        near-death
replace_with    near death experience
Name: 2228.0, dtype: object
key             ms-smart
replace_with       smart
Name: 2232.0, dtype: object
key                         self-injury
replace_with    Self injurious behavior
Name: 2236.0, dtype: object
key             heart failure-at1a
replace_with         heart failure
Name: 2258.0, dtype: object
key                                   fear-avoidance
replace_with    Fear-related avoidance of activities
Name: 2317.0, dtype: object
key   

Name: 4394.0, dtype: object
key             hiv-1-positive
replace_with               HIV
Name: 4423.0, dtype: object
key             technetium-stress
replace_with               stress
Name: 4441.0, dtype: object
key             papillomavirus-66
replace_with            Papilloma
Name: 4445.0, dtype: object
key             st-depression
replace_with       depression
Name: 4474.0, dtype: object
key             meningo-radiculitis
replace_with     Meningoradiculitis
Name: 4507.0, dtype: object
key             fibrosis-4
replace_with      fibrosis
Name: 4518.0, dtype: object
key             papillomavirus-56
replace_with            Papilloma
Name: 4593.0, dtype: object
key                   lqts-death
replace_with    Long QT syndrome
Name: 4603.0, dtype: object
key                                              lbbb-hf
replace_with    Left bundle branch block + heart failure
Name: 4625.0, dtype: object
key             non-stemi
replace_with    Non STEMI
Name: 4629.0, dtype: object
key     

key             hads-depression
replace_with         depression
Name: 6983.0, dtype: object
key             stress-mass-hr
replace_with            stress
Name: 7005.0, dtype: object
key             post-intracerebral haemorrhage
replace_with                       haemorrhage
Name: 7036.0, dtype: object
key             manic-depressive
replace_with    Manic depressive
Name: 7106.0, dtype: object
key             diarrhoea-predominant
replace_with                diarrhoea
Name: 7157.0, dtype: object
key             questionnaire-hepatitis
replace_with                  hepatitis
Name: 7164.0, dtype: object
key             papillomavirus-dna
replace_with             Papilloma
Name: 7220.0, dtype: object
key             infarction-carvedilol
replace_with               infarction
Name: 7267.0, dtype: object
key             hepatitis co-infection
replace_with                 hepatitis
Name: 7269.0, dtype: object
key             sars-cov
replace_with        sars
Name: 7287.0, dtype: object
key 

key             acute coronary syndromes
replace_with     acute coronary syndrome
Name: 89.0, dtype: object
key             gastrointestinal tuberculosis
replace_with                     tuberculosis
Name: 117.0, dtype: object
key             cardiac arrhythmias
replace_with     cardiac arrhythmia
Name: 118.0, dtype: object
key             kidney cancers
replace_with     kidney cancer
Name: 123.0, dtype: object
key             hemorrhagic strokes
replace_with     hemorrhagic stroke
Name: 148.0, dtype: object
key              bradyasystole
replace_with    bradyarrhytmia
Name: 185.0, dtype: object
key             breast cancers
replace_with     breast cancer
Name: 193.0, dtype: object
key             breast disorders
replace_with     breast disorder
Name: 210.0, dtype: object
key             behavioural disorders
replace_with     behavioural disorder
Name: 227.0, dtype: object
key             pain in limbs
replace_with     pain in limb
Name: 230.0, dtype: object
key             nodal blo

key             pseudocysts
replace_with     Pseudocyst
Name: 1386.0, dtype: object
key             erythrodysesthesia
replace_with           Dysesthesia
Name: 1419.0, dtype: object
key             paresthesias
replace_with     paresthesia
Name: 1434.0, dtype: object
key             colds
replace_with     cold
Name: 1445.0, dtype: object
key             minor bleeding
replace_with       Minor bleed
Name: 1453.0, dtype: object
key             glomerular filtration rate lower
replace_with      Glomerular filtration rate low
Name: 1455.0, dtype: object
key             poisonings
replace_with     poisoning
Name: 1501.0, dtype: object
key             exanthemas
replace_with     exanthema
Name: 1512.0, dtype: object
key             acute painful
replace_with       Acute pain
Name: 1513.0, dtype: object
key             depressionn
replace_with     depression
Name: 1536.0, dtype: object
key             biliary tract disorders
replace_with     biliary tract disorder
Name: 1543.0, dtype: object


Name: 2516.0, dtype: object
key             infarctions
replace_with     infarction
Name: 2541.0, dtype: object
key                       pneumocystis
replace_with    Pneumocystis pneumonia
Name: 2564.0, dtype: object
key             administration site reactions
replace_with     administration site reaction
Name: 2565.0, dtype: object
key             skin infections
replace_with     skin infection
Name: 2586.0, dtype: object
key             alcohol use disorders
replace_with     Alcohol use disorder
Name: 2588.0, dtype: object
key             internal herniation
replace_with             herniation
Name: 2616.0, dtype: object
key                 supraventricular arrhythmia
replace_with    Supraventricular arrhythmia NOS
Name: 2629.0, dtype: object
key             cardiac deaths
replace_with     cardiac death
Name: 2630.0, dtype: object
key             gastrointestinal disorders
replace_with     gastrointestinal disorder
Name: 2640.0, dtype: object
key             bleeding gums
replace_

key             hyperplastic polyps
replace_with                  polyp
Name: 4586.0, dtype: object
key             bacterial infections
replace_with     bacterial infection
Name: 4595.0, dtype: object
key               vulvovaginal lesion
replace_with    Vulvovaginal disorder
Name: 4599.0, dtype: object
key                    denervation
replace_with    denervation atropy
Name: 4600.0, dtype: object
key             depressive disorders
replace_with     depressive disorder
Name: 4610.0, dtype: object
key             noncardiac deaths
replace_with                death
Name: 4623.0, dtype: object
key             osteoporotic fractures
replace_with     osteoporotic fracture
Name: 4631.0, dtype: object
key             perinatal disorders
replace_with     perinatal disorder
Name: 4639.0, dtype: object
key             local reactions
replace_with     local reaction
Name: 4688.0, dtype: object
key             skin disorders
replace_with     skin disorder
Name: 4690.0, dtype: object
key       

### 여러 부작용이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [23]:
def separate_unique_ads(ads):
    additional_ads = []

    for ind, row in ads[ads.refined.str.find('+')>=0].iterrows():
        processed = list(map(lambda x: x.strip(), row['refined'].split('+')))

        if len(processed)>0:
            ads.loc[ind].refined = processed[0]
            for pro in processed[1:]:

                additional_ads.append({'name':row['name'], 'refined':pro})

    for ind, row in ads[ads.refined.str.find(',')>=0].iterrows():
        processed = list(map(lambda x: x.strip(), row['refined'].split(',')))

        if len(processed)>0:
            ads.loc[ind].refined = processed[0]
            for pro in processed[1:]:

                additional_ads.append({'name':row['name'], 'refined':pro})


    ads = ads.append(pd.DataFrame(additional_ads))
    ads.drop_duplicates(inplace=True)
    return ads

ads = separate_unique_ads(ads)

In [24]:
ads.shape

(3999, 2)

### 추가 전처리가 필요한 약물 갯수

In [25]:
len(ads[(ads.refined=='')])

81

In [26]:
def get_only_string(text):
    try:
        float(text)
        return False
    except ValueError:
        return True
    
def remove_comparison(name, spliter):
    arrs = list(filter(lambda x: get_only_string(x.strip().replace('%', '')) and x.strip(), name.split(spliter)))
    if len(arrs)>0:
        return arrs
    else:
        return None

for ind, row in ads.loc[ads.name.str.find('<')>=0].iterrows():
    processed = remove_comparison(row['name'], '<')
    if processed is None:
        ads.drop(ind, inplace=True)
    else:
        ads.loc[ind].refined = ''.join(processed)
# comp_ads.refined = comp_ads.name.map(lambda x: remove_comparison(x))

In [27]:
ads.loc[ads.name.str.find('<')>=0]

,name,refined
1587,values<25,values
1873,dyslipidemia<5,dyslipidemia
3477,diabetes<5,diabetes
3659,values<15,values
4040,pneumonia<1,pneumonia
4713,binge drinking<1,binge drinking


여러 부작용이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [28]:
additional_ads = []

for ind, row in ads.loc[ads.name.str.find('>')>=0].iterrows():
    processed = remove_comparison(row['name'], '>')
    if processed is None:
        ads.drop(ind, inplace=True)
    else:
        ads.loc[ind].refined = processed[0]
        for pro in processed[1:]:
            print(pro)
            additional_ads.append({'name':row['name'], 'refined':pro})

ads = ads.append(pd.DataFrame(additional_ads))
ads.drop_duplicates(inplace=True)

healthy
healthy


In [29]:
ads.shape

(4001, 2)

### 앞 뒤의 -, . 제거

In [30]:
remove_list = ['.', '-related', '-', ':', '+', '–', ';', '%', 'time', 'invariably', '||']
def preprocess(name):
    if name[0] in remove_list:
        name = name[1:len(name)]
    
    if name[len(name)-1] in remove_list:
        name = name[0:len(name)-1]
    return name
    
ads.name = ads.name.map(lambda x: preprocess(x))

In [31]:
remove_exact = ["electrocardiogram changes", "discharged", "historyshock", "alternans", "nonmelanoma skin", "nonmelanoma", "values",
               "statistics", 'actuarial', 'dangerous', 'shockable', 'presence of q waves', 'australia', "topical", "systemic",
               "familial", "fluor", "variations", "towards", "slaughter", "prestroke", "variation", "adjusting", "storage", 
                "subscale", "approached", "reached", "discharges", "accidental", "morale", "australian", "teache", "accidents", 
                "overfitting", "persisting", "confusional", "typically", "existing", "scales", "yards", "typical", "australi", 
                "biomass", "onwards", "subscales", "univariatep", "unrestricted", "rewards", "variant", "mutivariate", "vegetarian", 
                "intolerance", "bivariate", "antibiotics", "adjusted", "cystatin", "hazards", "intense", "anginal", "elispot", 
                "blinded", "pregnant", "pressure", "antipsychotic", "deaths", "fluoropyrimidine", "tolerance", "variance", "cavity", 
                "fluorouracil", "transapical", "resting", "varia", "edwards", "fluorodeoxyglucose", "homocysteine", "teriflunomide", 
                "defibrillation", 'sudden death—not', 'burn—hot', 'beverages'
               ]
for i, row in ads[ads.refined==''].iterrows():
    if row['name'].strip().lower() in remove_exact:
        ads.drop(i, inplace=True)

In [32]:
apply_dictionary()

ads.drop_duplicates(inplace=True)

/home/grace/.local/lib/python3.5/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
ads.shape

(3897, 2)

In [34]:
ads[ads.refined==''].shape

(3, 2)

In [35]:
ads[ads.refined=='']

,name,refined
1304,psychopathy,
1540,bleedings,
4171,stress,


In [36]:
ads = replace_with_manual_curation(ads)

key              anuloaortic ectasia
replace_with    annuloaortic ectasia
Name: 0.0, dtype: object
key             epispadias-classical
replace_with              epispadias
Name: 14.0, dtype: object
key             broncho-pneumonia
replace_with    broncho pneumonia
Name: 24.0, dtype: object
key             cardio-myopathy
replace_with     cardiomyopathy
Name: 40.0, dtype: object
key               hypertension-stroke
replace_with    Hypertension + stroke
Name: 50.0, dtype: object
key             mgh-adhd
replace_with        ADHD
Name: 64.0, dtype: object
key             heat-shock
replace_with         shock
Name: 77.0, dtype: object
key             neoplasm-associated
replace_with               neoplasm
Name: 86.0, dtype: object
key               neoangiogenesis-inflammation
replace_with    Neoangiogenesis + inflammation
Name: 114.0, dtype: object
key               dementia-motor neuron disease
replace_with    Dementia + motor neuron disease
Name: 132.0, dtype: object
key             v

key             tetanus-toxoid
replace_with           tetanus
Name: 2010.0, dtype: object
key             hads-anxiety
replace_with         anxiety
Name: 2029.0, dtype: object
key              aneurysms-osteoarthritis
replace_with    Aneurysm + ostemarthritis
Name: 2036.0, dtype: object
key               apnea–hypopnea
replace_with    apnea + hypopnea
Name: 2095.0, dtype: object
key             hypertension-ctd
replace_with        hypertension
Name: 2100.0, dtype: object
key             influenza-infected
replace_with             influenza
Name: 2110.0, dtype: object
key             diarrhoea-prevalence
replace_with               diarrhoea
Name: 2125.0, dtype: object
key             tension-type
replace_with         tension
Name: 2133.0, dtype: object
key             road-traffic accident
replace_with    Road traffic accident
Name: 2152.0, dtype: object
key             wisckott-aldrich syndrome
replace_with     Wiskott-Aldrich syndrome
Name: 2156.0, dtype: object
key             anxiet

/home/grace/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


key             sleep apnoea-hypopnoea
replace_with         Apnea + hypopnoea
Name: 4112.0, dtype: object
key             uab-adhd
replace_with        adhd
Name: 4173.0, dtype: object
key             non-eosinophilic asthma
replace_with                     asthma
Name: 4189.0, dtype: object
key             hypoxia-inducible
replace_with              hypoxia
Name: 4195.0, dtype: object
key             infarction-metoprolol
replace_with               infarction
Name: 4223.0, dtype: object
key             left-ventricular hypertrophy
replace_with    Left ventricular hypertrophy
Name: 4230.0, dtype: object
key             pre-hypertensive
replace_with     Prehypertension
Name: 4315.0, dtype: object
key             dilated cardiomyopathy-3
replace_with              cardiomyopathy
Name: 4352.0, dtype: object
key             hyperinsulinemic-euglycemic
replace_with               Hyperinsulinemia
Name: 4370.0, dtype: object
key             apoptosis-inducing
replace_with             apoptosis


key             scale-anxiety
replace_with          anxiety
Name: 8437.0, dtype: object
key             non-major bleeding
replace_with              bleeding
Name: 8474.0, dtype: object
key               hypertension-heart failure
replace_with    Hypertension + heart failure
Name: 8479.0, dtype: object
key             fall-from
replace_with         fall
Name: 8495.0, dtype: object
key             brain-death
replace_with    Brain death
Name: 8516.0, dtype: object
key                      respiratory-distress
replace_with    Respiratory distress syndrome
Name: 8518.0, dtype: object
key               sinusitis-otitis
replace_with    Sinusitis + otitis
Name: 8565.0, dtype: object
key             infarction-placebo
replace_with            infarction
Name: 8569.0, dtype: object
key               confusion-ataxia
replace_with    Confusion + Ataxia
Name: 8632.0, dtype: object
key             major-stroke
replace_with          stroke
Name: 8652.0, dtype: object
key             frontotemporal d

key             cysts
replace_with     cyst
Name: 937.0, dtype: object
key             trachomatous
replace_with        trachoma
Name: 957.0, dtype: object
key             scars
replace_with     scar
Name: 970.0, dtype: object
key             respiratory tract infections
replace_with     respiratory tract infection
Name: 974.0, dtype: object
key             brain scan abnormalities
replace_with         Brain scan abnormal
Name: 999.0, dtype: object
key             sexually transmitted diseases
replace_with     sexually transmitted disease
Name: 1001.0, dtype: object
key             tachyarrhythmias
replace_with     tachyarrhythmia
Name: 1047.0, dtype: object
key             procedural complications
replace_with     procedural complication
Name: 1048.0, dtype: object
key             restenosis
replace_with      stenosis
Name: 1049.0, dtype: object
key             neoplasms
replace_with     neoplasm
Name: 1053.0, dtype: object
key             middle cerebral artery infarction
replace_wit

key             windy
replace_with    windy
Name: 2190.0, dtype: object
key             abdominal cramping
replace_with       abdominal cramp
Name: 2193.0, dtype: object
key             burns
replace_with     burn
Name: 2198.0, dtype: object
key             instillation site reactions
replace_with     Instillation site reaction
Name: 2220.0, dtype: object
key             nervous system disorders
replace_with     nervous system disorder
Name: 2265.0, dtype: object
key             cramping
replace_with       cramp
Name: 2282.0, dtype: object
key             rashes
replace_with      rash
Name: 2294.0, dtype: object
key             preterm birth
replace_with    Preterm labor
Name: 2304.0, dtype: object
key             shocks
replace_with     shock
Name: 2311.0, dtype: object
key             polycystic kidney disease
replace_with            Polycystic kidney
Name: 2335.0, dtype: object
key                 endocrinopathy
replace_with    Endocrine disorder
Name: 2359.0, dtype: object
key     

key             headaches
replace_with     headache
Name: 3778.0, dtype: object
key             atrial arrhythmias
replace_with     atrial arrhythmia
Name: 3783.0, dtype: object
key             fractures
replace_with     fracture
Name: 3785.0, dtype: object
key             palpitationsor
replace_with       palpitation
Name: 3814.0, dtype: object
key             herniation
replace_with    herniation
Name: 3823.0, dtype: object
key             chronic liver diseases
replace_with     chronic liver disease
Name: 3828.0, dtype: object
key             headcache
replace_with     headache
Name: 3833.0, dtype: object
key             ?쐇nfusion site reactions??
replace_with        Infusion site reaction
Name: 3838.0, dtype: object
key              cardiorespiratory arrest
replace_with    Cardio-respiratory arrest
Name: 3842.0, dtype: object
key             reinfarction
replace_with      infarction
Name: 3845.0, dtype: object
key             congenital malformations
replace_with          Congenita

In [37]:
ads['llt_code'] = ''
ads['pt_code'] = ''
ads.head()

,name,refined,llt_code,pt_code
0,odynophagia,odynophagia,,
2,myalgia,myalgia,,
3,seizure,seizure,,
4,aphthous ulcer,aphthous ulcer,,
5,proliferative diabetic retinopathy,proliferative diabetic retinopathy,,


In [38]:
ads = separate_unique_ads(ads)

/home/grace/.local/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [39]:
for ind, grp in ads.groupby('refined'):
#     print(ind)
    found = dictionary.loc[dictionary.t_ad==ind.strip()]
    if len(found)>0:
#         print(found['cui_id'].values[0])
        ads.ix[grp.index.values, 'llt_code'] = found['llt_code'].values[0]
        ads.ix[grp.index.values, 'pt_code'] = found['pt_code'].values[0]

/home/grace/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/grace/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [40]:
# ads[(ads.llt_code=='') | (ads.pt_code=='')].to_csv('./0311_ad_need_tobe_refined.csv', sep=',', index=False)
ads[(ads.llt_code=='') | (ads.pt_code=='')]

,llt_code,name,pt_code,refined
58,,withdrawn,,withdrawn
183,,bradyasystole,,bradyarrhytmia
325,,non-ischaemia-related,,non-ischaemia
343,,illness-related,,illness
496,,pain gastrointestinal-related,,pain gastrointestinal
511,,wind,,wind
625,,lues,,lues
762,,serum-sickness-like,,serum-sickness
980,,in-stent restenosis,,stent retenosis
1587,,values<25,,values


### 추가 전처리 필요없는 부작용들은  refined로 입력

In [ ]:
# remove_list.extend([')', '('])
# spe_chars = dict((re.escape(str(k)), ' ') for k in remove_list)
# sep_pattern = re.compile("|".join(spe_chars.keys()))
# int_pattern = re.compile('[0-9]')

# for ind, row in ads.iterrows():
#     refined = sep_pattern.sub(lambda m: spe_chars[re.escape(m.group(0))], row['name'])
#     int_refined = int_pattern.search(row['name'])
    
#     if refined.strip() == '':
#         ads.drop(ind, inplace=True)
#     elif row['name'].strip() == refined.strip() and int_refined is None:
#         ads.loc[ind]['refined'] = refined
#         print(row['name'])
#         print(ads.loc[ind]['refined'])
#     print('===============================')

In [ ]:
# ads.loc[ads.refined.str.len()>0].shape

In [ ]:
# ads.reset_index(drop=True)

In [ ]:
ads[ads.name.str.find('sudden death')>=0]

## 정체된 목록입니다.

In [ ]:
ads.loc[ads.refined.str.strip()!=''].to_csv('./refined_ads_0226.csv', sep=',')

## 선생님께 확인 요청을 드리고 싶은 부분

In [ ]:
ads[ads['refined'] == ''].to_csv('./need_to_refined_ads_0226.csv', sep=',')